# Gathering and cleaning data

The goal of this project is to predict the number of Citibikes rides in NYC starting at a given station during a given hour depending on the weather.

The Citibikes datasets used in this project (July 2021 - June 2023) contain the following columns:

- `ride_id`
- `rideable_type`: electric_bike, classic_bike
- `started_at`
- `ended at`
- `start_station_name`
- `start_station_id`
- `end_station_name`
- `end_station_id`
- `start_lat` 
- `start_lng`
- `end_lat`
- `end_long`
- `rider_type`: member/casual

We will focus on the following columns: `started_at`, `start_station_id`, `start_station_name`, `start_lat`, `start_lng`. 

From `started_at`, we will create the following new column:
- `weekday`

Each row in the dataset is a ride, so we will group by these features and count the number of rows to get the number of rides per station, per hour.
- `count`

In [1]:
import pandas as pd
import polars as pl
import glob
import yaml
from functions import yearly_clean_data, concat_dfs, get_station_data
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pl.Config(fmt_str_lengths=50)

In [2]:
try:
    with open("../config.yaml", 'r') as file:
        config = yaml.safe_load(file)
except Exception as e:
    print('Error reading config file')

## Getting Data

### Bike trip data

In [3]:
data_2021 = yearly_clean_data('../data/raw/2021*.csv')

The total rows in the entire df is 521918
Total rows: 76235, total count 521918
total rows: 76235, total count in the concated df is 521918


In [4]:
data_2021.head()

started_at_rounded,weekday,start_station_id,count
datetime[μs],u32,str,u32
2021-07-01 00:00:00,4,"""4993.13""",1
2021-07-01 00:00:00,4,"""5065.12""",1
2021-07-01 00:00:00,4,"""5114.06""",1
2021-07-01 00:00:00,4,"""5137.11""",1
2021-07-01 00:00:00,4,"""5137.13""",1


In [5]:
data_2022 = yearly_clean_data('../data/raw/2022*.csv')

The total rows in the entire df is 949054
Total rows: 158916, total count 949054
total rows: 158916, total count in the concated df is 949054


In [6]:
data_2022.head()

started_at_rounded,weekday,start_station_id,count
datetime[μs],u32,str,u32
2022-01-01 00:00:00,6,"""4889.06""",1
2022-01-01 00:00:00,6,"""4953.04""",1
2022-01-01 00:00:00,6,"""4993.13""",1
2022-01-01 00:00:00,6,"""5001.08""",4
2022-01-01 00:00:00,6,"""5065.04""",3


In [7]:
data_2023 = yearly_clean_data('../data/raw/2023*.csv')

The total rows in the entire df is 450987
Total rows: 85992, total count 450987
total rows: 85992, total count in the concated df is 450987


In [8]:
data_2023.head()

started_at_rounded,weekday,start_station_id,count
datetime[μs],u32,str,u32
2023-01-01 00:00:00,7,"""4889.06""",1
2023-01-01 00:00:00,7,"""4953.04""",3
2023-01-01 00:00:00,7,"""4962.08""",1
2023-01-01 00:00:00,7,"""5001.08""",3
2023-01-01 00:00:00,7,"""5033.01""",1


In [9]:
all_data_lst = [data_2021, data_2022, data_2023]

In [10]:
data = concat_dfs(all_data_lst)

total rows: 321143, total count in the concated df is 1921959


In [11]:
data.head()

started_at_rounded,weekday,start_station_id,count
datetime[μs],u32,str,u32
2021-07-01 00:00:00,4,"""4993.13""",1
2021-07-01 00:00:00,4,"""5065.12""",1
2021-07-01 00:00:00,4,"""5114.06""",1
2021-07-01 00:00:00,4,"""5137.11""",1
2021-07-01 00:00:00,4,"""5137.13""",1


In [12]:
data.write_csv('../data/cleaned/all_data.csv')

### Getting unique station id information
There are station ids that have different lat and long values for them. We will get station names and one set of lat and long for each station id.

In [13]:
station_info = get_station_data('../data/raw/2023*.csv')

In [14]:
station_info.head()

start_station_id,start_station_name,start_lat,start_lng
str,str,f64,f64
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342
"""4889.06""","""Bus Slip & State St""",40.701907,-74.013942
"""4953.04""","""South St & Gouverneur Ln""",40.703554,-74.006702
"""4962.01""","""Broadway & Battery Pl""",40.704633,-74.013617
"""4962.02""","""Whitehall St & Bridge St""",40.703748,-74.013133


In [15]:
station_info.write_csv('../data/cleaned/station_info.csv')

### Weather data

The historical weather data was obtained from [Open-Meteo](https://open-meteo.com). 

Historical weather data was downloaded for the date range of the Citibike data, July 1, 2021 through June 30, 2023.

In [31]:
weather = pl.read_csv(config["data"]["weather"],
                      has_header=False,
                      skip_rows = 4,
                      new_columns=['time', 'temperature_f', 'precipitation_in', 'windspeed_mph'],
                     dtypes={'time':pl.Datetime, 'temperature_F':pl.Float64, 'precipitation_in':pl.Float64, 'windspeed_mph':pl.Float64})

In [32]:
weather.head()

time,temperature_f,precipitation_in,windspeed_mph
datetime[μs],f64,f64,f64
2021-07-01 00:00:00,78.6,0.0,3.2
2021-07-01 01:00:00,77.5,0.0,2.5
2021-07-01 02:00:00,76.4,0.0,2.0
2021-07-01 03:00:00,74.9,0.0,3.2
2021-07-01 04:00:00,73.9,0.0,3.5


Dataset was checked for missing rows, and there are none.

In [40]:
weather.write_csv('../data/cleaned/weather_clean.csv')

## Concating rides, station info, weather data

In [41]:
data_station = data.join(station_info, how = 'left', on='start_station_id')

In [42]:
data_station.head()

started_at_rounded,weekday,start_station_id,count,start_station_name,start_lat,start_lng
datetime[μs],u32,str,u32,str,f64,f64
2021-07-01 00:00:00,4,"""4993.13""",1,"""Old Slip & South St""",40.703367,-74.007868
2021-07-01 00:00:00,4,"""5065.12""",1,"""William St & Pine St""",40.707179,-74.008873
2021-07-01 00:00:00,4,"""5114.06""",1,"""West Thames St""",40.708347,-74.017134
2021-07-01 00:00:00,4,"""5137.11""",1,"""Fulton St & William St""",40.709601,-74.006551
2021-07-01 00:00:00,4,"""5137.13""",1,"""Spruce St & Gold St""",40.710323,-74.004323


In [43]:
data_station_weather = data_station.join(weather, how='left', left_on='started_at_rounded', right_on='time')

In [44]:
data_station_weather.tail(25)

started_at_rounded,weekday,start_station_id,count,start_station_name,start_lat,start_lng,temperature_f,precipitation_in,windspeed_mph
datetime[μs],u32,str,u32,str,f64,f64,f64,f64,f64
2023-06-30 23:00:00,5,"""5137.11""",7,"""Fulton St & William St""",40.709601,-74.006551,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5145.02""",2,"""Albany St & Greenwich St""",40.708724,-74.013093,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5175.08""",4,"""Fulton St & Broadway""",40.711187,-74.009484,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5184.08""",7,"""West St & Liberty St""",40.711444,-74.014847,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5207.01""",8,"""Centre St & Chambers St""",40.712785,-74.004562,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5216.04""",1,"""Barclay St & Church St""",40.712912,-74.010202,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5216.06""",9,"""Vesey St & Church St""",40.712455,-74.010822,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5288.08""",1,"""Park Pl & Church St""",40.713089,-74.009329,68.8,0.0,4.7
2023-06-30 23:00:00,5,"""5288.09""",4,"""Warren St & W Broadway""",40.71474,-74.009106,68.8,0.0,4.7


Due to the rounding function, several dates were cast to July 1, 2023. We will drop these values to be consistent with the rounding.

In [45]:
data_station_weather_clean = data_station_weather.filter(pl.col('started_at_rounded') < datetime(2023, 7, 1))

In [46]:
data_station_weather_clean.null_count()

started_at_rounded,weekday,start_station_id,count,start_station_name,start_lat,start_lng,temperature_f,precipitation_in,windspeed_mph
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0


In [47]:
data_station_weather_clean.write_csv('../data/cleaned/data_station_weather_clean.csv')

## Getting forecast data

The forecasted weather data was obtained from [Open-Meteo](https://open-meteo.com) through their open API. 

The forecast is the 3-day forecast for NYC.

In [25]:
import json
import requests

In [48]:
api_call = "https://api.open-meteo.com/v1/forecast?latitude=40.7143&longitude=-74.006&hourly=temperature_2m,precipitation,windspeed_10m&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch&timezone=America%2FNew_York&forecast_days=3"

response = requests.get(api_call)
print(response.status_code)
results = response.json()
results

200


{'latitude': 40.710335,
 'longitude': -73.99307,
 'generationtime_ms': 0.45096874237060547,
 'utc_offset_seconds': -14400,
 'timezone': 'America/New_York',
 'timezone_abbreviation': 'EDT',
 'elevation': 51.0,
 'hourly_units': {'time': 'iso8601',
  'temperature_2m': '°F',
  'precipitation': 'inch',
  'windspeed_10m': 'mp/h'},
 'hourly': {'time': ['2023-08-04T00:00',
   '2023-08-04T01:00',
   '2023-08-04T02:00',
   '2023-08-04T03:00',
   '2023-08-04T04:00',
   '2023-08-04T05:00',
   '2023-08-04T06:00',
   '2023-08-04T07:00',
   '2023-08-04T08:00',
   '2023-08-04T09:00',
   '2023-08-04T10:00',
   '2023-08-04T11:00',
   '2023-08-04T12:00',
   '2023-08-04T13:00',
   '2023-08-04T14:00',
   '2023-08-04T15:00',
   '2023-08-04T16:00',
   '2023-08-04T17:00',
   '2023-08-04T18:00',
   '2023-08-04T19:00',
   '2023-08-04T20:00',
   '2023-08-04T21:00',
   '2023-08-04T22:00',
   '2023-08-04T23:00',
   '2023-08-05T00:00',
   '2023-08-05T01:00',
   '2023-08-05T02:00',
   '2023-08-05T03:00',
   '2023-08

In [49]:
forecast_3day = pd.DataFrame(results['hourly'])

In [50]:
forecast_3day

,time,temperature_2m,precipitation,windspeed_10m
0,2023-08-04T00:00,72.0,0.0,9.4
1,2023-08-04T01:00,71.2,0.0,10.5
2,2023-08-04T02:00,73.1,0.0,9.5
3,2023-08-04T03:00,69.8,0.0,9.0
4,2023-08-04T04:00,70.4,0.0,8.2
5,2023-08-04T05:00,69.5,0.0,8.7
6,2023-08-04T06:00,70.3,0.0,10.4
7,2023-08-04T07:00,70.8,0.0,10.3
8,2023-08-04T08:00,72.3,0.0,10.7
9,2023-08-04T09:00,73.3,0.0,10.2


In [51]:
forecast_3day.to_csv("../data/raw/forecast_3day.csv", index=False)

In [52]:
forecast_3day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   time            72 non-null     object 
 1   temperature_2m  72 non-null     float64
 2   precipitation   72 non-null     float64
 3   windspeed_10m   72 non-null     float64
dtypes: float64(3), object(1)
memory usage: 2.4+ KB


In [60]:
forecast_3day_clean = forecast_3day.copy()
forecast_3day_clean['time']=pd.to_datetime(forecast_3day_clean['time'], format="%Y-%m-%dT%H:%M").dt.strftime('%Y-%m-%d %H:%M:%S')

forecast_3day_clean.columns = ['started_at_rounded', 'temperature_f', 'precipitation_in', 'windspeed_mph']

In [61]:
forecast_3day_clean

,started_at_rounded,temperature_f,precipitation_in,windspeed_mph
0,2023-08-04 00:00:00,72.0,0.0,9.4
1,2023-08-04 01:00:00,71.2,0.0,10.5
2,2023-08-04 02:00:00,73.1,0.0,9.5
3,2023-08-04 03:00:00,69.8,0.0,9.0
4,2023-08-04 04:00:00,70.4,0.0,8.2
5,2023-08-04 05:00:00,69.5,0.0,8.7
6,2023-08-04 06:00:00,70.3,0.0,10.4
7,2023-08-04 07:00:00,70.8,0.0,10.3
8,2023-08-04 08:00:00,72.3,0.0,10.7
9,2023-08-04 09:00:00,73.3,0.0,10.2


In [55]:
forecast_3day_clean.shape[0]

72

In [62]:
forecast_3day_clean.to_csv('../data/cleaned/forecast_3day_clean.csv', index=False)

### Combine station info with forecast data to use in the predictions

In [63]:
forecast_pl = pl.from_pandas(forecast_3day_clean)
prediction_data = station_info.join(forecast_pl, how="cross")

In [64]:
prediction_data.shape[0]

2160

In [65]:
prediction_data

start_station_id,start_station_name,start_lat,start_lng,started_at_rounded,temperature_f,precipitation_in,windspeed_mph
str,str,f64,f64,str,f64,f64,f64
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 00:00:00""",72.0,0.0,9.4
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 01:00:00""",71.2,0.0,10.5
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 02:00:00""",73.1,0.0,9.5
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 03:00:00""",69.8,0.0,9.0
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 04:00:00""",70.4,0.0,8.2
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 05:00:00""",69.5,0.0,8.7
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 06:00:00""",70.3,0.0,10.4
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 07:00:00""",70.8,0.0,10.3
"""4846.01""","""South St & Whitehall St""",40.701221,-74.012342,"""2023-08-04 08:00:00""",72.3,0.0,10.7


In [66]:
prediction_data.write_csv('../data/cleaned/prediction_data.csv')